# Week2: statsmodels를 이용한 회귀분석

1주차에 생성한 선형 회귀 모델을 `statsmodels` 라이브러리를 사용하여 더 깊이 분석합니다.
각 변수(Feature)의 통계적 유의성을 확인하고, 모델의 설명력을 평가합니다.

## 1. 데이터 준비 및 statsmodels 회귀모델 생성

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
import statsmodels.api as sm

# scikit-learn 1.2 버전부터 load_boston 함수가 제거되었습니다.
# fetch_openml을 사용하여 동일한 데이터셋을 불러옵니다.
boston = fetch_openml(name='boston', version=1, as_frame=True, parser='liac-arff')
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['PRICE'] = boston.target

# 특성(X)과 타겟(y) 분리
X = df.drop('PRICE', axis=1)
y = df['PRICE']

# 데이터 타입을 numeric으로 변환하여 object dtype 오류 해결
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

# NaN 값이 있다면 제거
if X.isnull().any().any() or y.isnull().any():
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]

# statsmodels OLS 모델을 위해 X 변수에 상수항(intercept) 추가
X_const = sm.add_constant(X)

# OLS 모델 적합
model = sm.OLS(y, X_const)
results = model.fit()


## 2. 회귀분석 리포트 출력 및 해석

In [4]:
# 회귀분석 결과 요약 출력
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 15 Sep 2025   Prob (F-statistic):          6.72e-135
Time:                        23:27:43   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.4595      5.103      7.144      0.0

## 3. 결과 해석

### 각 변수의 p-value (P>|t|)
p-value는 각 변수의 계수가 통계적으로 얼마나 유의미한지를 나타내는 지표입니다. 일반적으로 p-value가 0.05보다 작을 때 (유의수준 5% 기준), 해당 변수는 종속변수(PRICE)에 유의미한 영향을 미친다고 해석합니다.

- **유의미한 변수 (p-value < 0.05)**: `const`, `CRIM`, `ZN`, `CHAS`, `NOX`, `RM`, `DIS`, `RAD`, `TAX`, `PTRATIO`, `B`, `LSTAT`
  - 이 변수들은 주택 가격 예측에 통계적으로 중요한 역할을 합니다.
- **유의미하지 않은 변수 (p-value >= 0.05)**: `INDUS`, `AGE`
  - `INDUS`(비소매상업지역 토지 비율)와 `AGE`(오래된 주택 비율)는 이번 모델에서 주택 가격에 미치는 영향이 통계적으로 유의미하지 않다고 나타났습니다. 즉, 이 변수들이 변하더라도 주택 가격의 변동을 설명하기 어렵다는 의미입니다.

### R-squared / Adj. R-squared 값

- **R-squared (결정계수)**: 0.741
  - 모델의 설명력을 나타내는 지표로, 0과 1 사이의 값을 가집니다. 1에 가까울수록 모델이 데이터를 잘 설명한다는 의미입니다. R-squared가 0.741이라는 것은 이 모델이 **주택 가격 변동의 약 74.1%를 설명**한다는 것을 의미합니다. 비교적 높은 설명력을 가진다고 볼 수 있습니다.

- **Adj. R-squared (조정된 결정계수)**: 0.734
  - 독립변수의 개수가 늘어날수록 R-squared 값이 증가하는 경향을 보정해주는 지표입니다. 불필요한 변수가 추가될 경우 이 값은 감소하므로, 서로 다른 모델을 비교할 때 더 적합한 척도입니다. R-squared 값과 큰 차이가 없으므로, 현재 모델에 불필요한 변수가 많지는 않다고 해석할 수 있습니다.

### 종합

이 모델은 보스턴 주택 가격 데이터의 변동성을 약 74% 설명할 수 있으며, 이는 꽤 준수한 성능이라고 생각됩니다. 대부분의 변수들이 통계적으로 유의미하게 나타나 주택 가격 예측에 기여하고 있음을 확인했습니다. 

다만, `INDUS`와 `AGE` 변수는 p-value가 높아 유의미하지 않은 것으로 나타났습니다. 모델의 성능을 더 개선하기 위해서는 이 변수들을 제외하고 모델을 다시 만들거나, 다른 변수와 조합하여 새로운 파생 변수를 만드는 방법을 고려해볼 수 있습니다. 또한, 데이터에 비선형 관계가 존재할 수 있으므로, 이를 반영할 수 있는 다항 회귀나 다른 고급 모델을 사용하면 설명력을 더 높일 수 있을 것입니다.